In [4]:
# Paruošia pūpsnių indeksus ML mokymui ir vertinimui.
# 
# Panaudojant pacientų įrašų vardų SubjCode sąrašus train_lst, validate_lst, test_lst iš disko,
# sukūriami atitinkami indeksų failai train_ind_lst, validate_ind_lst, test_ind_lst ir įrašomi į diską.
# Kartu su duomenimis ir all_beats_attr, jie naudojami klasifikatorių kūrimui. 
# 
# Nebalansuotas variantas.
# 
# Paruošti pūpsnių indeksų masyvai gali būti panaudoti tiesioginiam sekų 
# formavimui iš EKG įrašų (rec) skaičiavimo metu arba sekų Numpy masyvų formavimui
#  
# //////////////////////////////////////////////////////////////////////////////////////////////////////
#
import pandas as pd
import numpy as np
from numpy import loadtxt, savetxt
import matplotlib.pyplot as plt
from pathlib import Path
from icecream import ic
import json, time, sys

from zive_util_vu import create_dir
# from zive_util_vu import anotaciju_pasiskirstymas_v2, runtime
from zive_util_vu import runtime
from zive_util_vu import get_annotations_table, print_annotations_table


my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  MIT2ZIVE duomenų aplankas
db_folder = 'DUOM_VU'

# Aplankas su MIT2ZIVE EKG įrašais (.npy) ir anotacijomis (.json)
rec_folder = 'records_npy'

# Duomenų schemos aplanko pavadinimas
# SCHEMA_FOLDER = 'schema_NSV_50_nbal'

# Kas kiek išvedamas apdorotų duomenų skaičius
show_period = 100

# Anotacijoms priskirtos klasės
selected_beats = {'N':0, 'S':1, 'V':2, 'U':3}

# Sekos kanalų skaičius
n_channels = 1



# Train imties pacientai
# DS1 = [ 10000, 10001]

# Train imties pacientai
# DS1 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119, 122, 124, 201, 203, 205, 208, 209, 215, 220, 223,230]

# Validate imties pacientai
# DS2 = [10010, 10031, 10032, 10060, 10061]

# Validate imties pacientai
# DS2 = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202, 210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]

# Kiekis pūpsnių, kurie atmetami EKG įrašo pradžioje ir pabaigoje
beats_skiped = 10

# ///////////////////////////////////////////////////////////////////////////////

# Anotacijų sąrašas
annot_list = [key for key in selected_beats]

# swapped_selected_beats
swapped_selected_beats = {v: k for k, v in selected_beats.items()}

# Nuoroda į MIT2ZIVE duomenų aplanką
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su MIT2ZIVE EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, rec_folder)

# Nuskaitome ir suformuojame DS1Z
file_path = Path(rec_dir, 'train_subjcode_lst.csv')
DSZ1 = list(np.loadtxt(file_path, delimiter=',', dtype="int"))
print(DSZ1)

# Nuskaitome ir suformuojame DS3Z
file_path = Path(rec_dir, 'test_subjcode_lst.csv')
DSZ3 = list(np.loadtxt(file_path, delimiter=',', dtype="int"))
print(DSZ3)


# Nuoroda į duomenų schemos aplanką ir vertinimo imties sekų indeksų failas
all_beats_attr_fname = 'all_beats_attr_z.csv'

# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)

fs = info_create['fs'] # diskretizavimo dažnumas
SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas

# Klasių skaičius
n_classes = len(selected_beats)

print("\nParuošia EKG pūpsnių indeksus ML mokymui ir vertinimui")
print("Nebalansuotas variantas")

print("\nBendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("Zive EKG įrašų aplankas: ", db_folder)

print("Diskretizavimo dažnis: ", fs)
print("\nPacientų įrašų sąrašas:\n",SubjCodes)
# print("Pacientų sąrašas mokymui:", DS1)
# print("Pacientų sąrašas vertinimui:", DS2)

# print("\nDuomenų schemos aplankas: ", SCHEMA_PATH)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))
print('Pūpsnių atributų masyvo failas:', all_beats_attr_fname)

print("\nParametrai pūpsnių kiekiui apriboti")
print(f"beats_skiped: {beats_skiped}")
print(f"Kiekviename EKG įraše praleisti {beats_skiped} pirmi ir {beats_skiped} paskutiniai pūpsniai")



OS in my system :  win32
[10010, 10020, 10030, 10050, 10060, 10070, 10011, 10080, 10031, 10021, 10022]
[10000, 10040, 10041, 10042, 10043, 10090, 10100, 10110, 10120, 10130]

Paruošia EKG pūpsnių indeksus ML mokymui ir vertinimui
Nebalansuotas variantas

Bendras Zive duomenų aplankas:  D:\DI\Data\MIT&ZIVE\VU
Zive EKG įrašų aplankas:  DUOM_VU
Diskretizavimo dažnis:  200

Pacientų įrašų sąrašas:
 [10000, 10010, 10020, 10030, 10040, 10041, 10050, 10042, 10060, 10043, 10070, 10011, 10080, 10031, 10090, 10100, 10110, 10021, 10120, 10022, 10130]
Klasifikavimo schema: {'N': 0, 'S': 1, 'V': 2, 'U': 3}
Klasių skaičius: 4
Pūpsnių atributų masyvo failas: all_beats_attr_z.csv

Parametrai pūpsnių kiekiui apriboti
beats_skiped: 10
Kiekviename EKG įraše praleisti 10 pirmi ir 10 paskutiniai pūpsniai


In [5]:

pd.set_option("display.max_rows", 1000, "display.max_columns", 19)
pd.set_option('display.width', 1000)

cols_pattern = ['N', 'S', 'V', 'F']
 
#  from mit2zive_util import create_set_from_rec
from sklearn.preprocessing import StandardScaler
from numpy import asarray
from zive_util_vu import split_SubjCode

start_time = time.time()

# Nuskaitome pūpsnių atributų freimą iš rec_dir aplanko
file_path = Path(rec_dir, all_beats_attr_fname)
all_beats_attr = pd.read_csv(file_path, index_col=0)

# Visų sekų indeksų sąrašas
all_beat_indices = all_beats_attr.index
# print(all_beat_indices)
# DS = DS1 + DS2
# all_ind_lst = []
# for userNr in DS:
#     selected_ind = all_beat_indices[all_beats_attr['userNr']==userNr]
#     all_ind_lst.extend(selected_ind.to_list())

print("\nAnotacijų pasiskirstymas visuose duomenyse\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr, cols_pattern=cols_pattern)
print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)

# Suformuojame sekų masyvus klasifikatoriaus mokymui ir vertinimui

# MOKYMO IMTIS

# Suformuojame mokymo imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus
# train_ind_lst = []
# for userNr in DSZ1:
#     selected_ind = all_beat_indices[all_beats_attr['userNr']==userNr]
#     selected_ind = selected_ind[beats_skiped:-beats_skiped]
#     train_ind_lst.extend(selected_ind.to_list())

# # Anotacijų pasiskirstymas mokymo duomenyse
# print("\nAnotacijų pasiskirstymas mokymo duomenyse\n")
# labels_table, labels_sums = anotaciju_pasiskirstymas_v2(all_beats_attr, ind_lst=train_ind_lst, cols_pattern=['N', 'S', 'V'])
# print(labels_table)
# print("\nTotal:", labels_sums['All'])

# # Įrašome mokymo imties sekų indeksų sąrašą į diską
# file_path = Path(SCHEMA_PATH, 'train_ind_lst.csv')
# np.savetxt(file_path, np.array(train_ind_lst), delimiter=',', fmt='%d')
# print("\nMokymo imties sekų indeksų sąrašas įrašytas į: ", file_path )

# Suformuojame mokymo sekų ir jų klasių numerių masyvus
# X_train, y_train = create_set_from_rec(rec_dir, all_beats_attr, train_ind_lst, window_left_side, window_right_side)
# print("\nMokymo imties dimensijos")
# print(f"{X_train.shape=} {y_train.shape=}")

# Įrašome mokymui skirtus masyvus į npz file diske
# file_path = Path(SCHEMA_PATH, 'train_set.npz')
# np.savez_compressed(file_path, X=X_train, y=y_train)
# print("\nMokymo sekų ir jų klasių numerių masyvas įrašytas: į ", file_path )

# Suformuojame imčių sekų indeksų sąrašus. Formuodami sąrašus
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus

# MOKYMO IMTIS

train_ind_lst = []
for SubjCode in DSZ1:
    userNr, recordingNr = split_SubjCode(SubjCode)
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recordingNr)]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    selected_ind_lst = selected_ind.to_list()
    print("selected_ind_lst parameters:")
    print(f"SubjCode: {SubjCode}  first elem: {selected_ind_lst[0]} last elem: {selected_ind_lst[-1]}  tot: {len(selected_ind_lst)}")
    train_ind_lst.extend(selected_ind_lst)

# Anotacijų pasiskirstymas vertinimo duomenyse
print("\nAnotacijų pasiskirstymas mokymo duomenyse\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr, ind_lst= train_ind_lst, cols_pattern=cols_pattern)
print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)

# Įrašome vertinimo imties sekų indeksų sąrašą į diską
file_path = Path(rec_dir, 'validate_ind_lst.csv')
np.savetxt(file_path, np.array(train_ind_lst), delimiter=',', fmt='%d')
print("\nMokymo imties sekų indeksų sąrašas įrašytas į: ", file_path )


# TESTINĖ IMTIS

test_ind_lst = []
for SubjCode in DSZ3:
    userNr, recordingNr = split_SubjCode(SubjCode)
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recordingNr)]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    selected_ind_lst = selected_ind.to_list()
    print("selected_ind_lst parameters:")
    print(f"SubjCode: {SubjCode}  first elem: {selected_ind_lst[0]} last elem: {selected_ind_lst[-1]}  tot: {len(selected_ind_lst)}")
    test_ind_lst.extend(selected_ind_lst)

# Anotacijų pasiskirstymas vertinimo duomenyse
print("\nAnotacijų pasiskirstymas testiniuose duomenyse\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr, ind_lst= test_ind_lst, cols_pattern=cols_pattern)
print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)

# Įrašome vertinimo imties sekų indeksų sąrašą į diską
file_path = Path(rec_dir, 'test_ind_lst.csv')
np.savetxt(file_path, np.array(test_ind_lst), delimiter=',', fmt='%d')
print("\nTestinės imties sekų indeksų sąrašas įrašytas į: ", file_path )


# VERTINIMO IMTIS
# Suformuojame vertinimo imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus

# validate_ind_lst = []
# for SubjCode in DS2:
#     userNr, recordingNr = split_SubjCode(SubjCode)
#     selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recordingNr)]
#     selected_ind = selected_ind[beats_skiped:-beats_skiped]
#     selected_ind_lst = selected_ind.to_list()
#     print("selected_ind_lst parameters:")
#     print(f"SubjCode: {SubjCode}  first elem: {selected_ind_lst[0]} last elem: {selected_ind_lst[-1]}  tot: {len(selected_ind_lst)}")
#     validate_ind_lst.extend(selected_ind_lst)

# # Anotacijų pasiskirstymas vertinimo duomenyse
# print("\nAnotacijų pasiskirstymas vertinimo duomenyse\n")
# labels_table, labels_sums = get_annotations_table(all_beats_attr, ind_lst= validate_ind_lst, cols_pattern=cols_pattern)
# print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)

# # Įrašome vertinimo imties sekų indeksų sąrašą į diską
# file_path = Path(rec_dir, 'validate_ind_lst.csv')
# np.savetxt(file_path, np.array(validate_ind_lst), delimiter=',', fmt='%d')
# print("\nVertinimo imties sekų indeksų sąrašas įrašytas į: ", file_path )

# Suformuojame mokymo sekų ir jų klasių numerių masyvus
# X_validate, y_validate = create_set_from_rec(rec_dir, all_beats_attr, validate_ind_lst, window_left_side, window_right_side)
# print("\nVertinimo imties dimensijos")
# print(f"{X_validate.shape=} {y_validate.shape=}")

# Įrašome vertinimui skirtus masyvus į npz file diske
# file_path = Path(SCHEMA_PATH, 'validate_set.npz')
# np.savez_compressed(file_path, X=X_validate, y=y_validate)
# print("\nVertinimo sekų ir jų klasių numerių masyvas įrašytas: į ", file_path )

end_time = time.time()
print('\n')
runtime(end_time-start_time)

# Suformuojame info_prep masyvą ir įrašome į diską

info = {
    'fs': fs,
    'SubjCodes':SubjCodes,
    'selected_beats':selected_beats,
    'beats_skiped':beats_skiped
    }

# file_path = Path(SCHEMA_PATH,'info_prep_tst.json')
# with open(file_path, 'w') as f:
#     json.dump(info, f)
    
# print("\nSchemos parametrai info.json yra aplanke: ", file_path, "\n" )

print("\nPabaiga.............")



Anotacijų pasiskirstymas visuose duomenyse

'N': 15025, 'S': 531, 'V': 548
Total:  16104
selected_ind_lst parameters:
SubjCode: 10010  first elem: 731 last elem: 1268  tot: 538
selected_ind_lst parameters:
SubjCode: 10020  first elem: 1289 last elem: 1839  tot: 551
selected_ind_lst parameters:
SubjCode: 10030  first elem: 1860 last elem: 2642  tot: 783
selected_ind_lst parameters:
SubjCode: 10050  first elem: 4725 last elem: 5491  tot: 767
selected_ind_lst parameters:
SubjCode: 10060  first elem: 6710 last elem: 7450  tot: 741
selected_ind_lst parameters:
SubjCode: 10070  first elem: 8455 last elem: 9133  tot: 679
selected_ind_lst parameters:
SubjCode: 10011  first elem: 9154 last elem: 9733  tot: 580
selected_ind_lst parameters:
SubjCode: 10080  first elem: 9754 last elem: 10545  tot: 792
selected_ind_lst parameters:
SubjCode: 10031  first elem: 10566 last elem: 11250  tot: 685
selected_ind_lst parameters:
SubjCode: 10021  first elem: 13451 last elem: 13971  tot: 521
selected_ind_lst